In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import nltk
import string
import matplotlib.pyplot as plt

In [2]:
fb_percentiles = pd.read_excel('/Users/harishprabhala/Downloads/martinchek-2012-2016-facebook-posts/fb_posts_percentiles.xlsx')

In [3]:
fb_percentiles['full_text'] = fb_percentiles['name'].map(str) + ' ' + fb_percentiles['message'].map(str) + ' ' + fb_percentiles['description'].map(str)

In [4]:
bins = [0,33,66,100]

In [5]:
group_names = ['low','med','high']

In [6]:
fb_percentiles_copy = fb_percentiles[['full_text','likes_count','shares_count','likes_percentile','shares_percentile']]

In [7]:
fb_percentiles_copy['likes_percentile'] = fb_percentiles_copy['likes_percentile']*100
fb_percentiles_copy['shares_percentile'] = fb_percentiles_copy['shares_percentile']*100

/Users/harishprabhala/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/harishprabhala/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [9]:
likes_categories = pd.cut(fb_percentiles_copy['likes_percentile'], bins, labels=group_names)
fb_percentiles_copy['likes_class'] = likes_categories

/Users/harishprabhala/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [10]:
fb_percentiles_copy['likes_class'] = fb_percentiles_copy['likes_class'].fillna('low')

/Users/harishprabhala/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
fb_percentiles_copy.head()

,full_text,likes_count,shares_count,likes_percentile,shares_percentile,likes_class
0,Chief Justice Roberts Responds to Judicial Eth...,61,12,5.6,9.8,low
1,"With Reservations, Obama Signs Act to Allow De...",120,171,22.4,75.1,low
2,Wishes For 2012 to Fall on Times Square Some p...,271,0,58.2,0.0,med
3,Mitt Romney Vows to Veto Dream Act if Presiden...,140,23,29.1,18.7,low
4,"NY Pharmacy Shootout Leaves Suspect, ATF Agent...",59,34,5.1,26.9,low


In [12]:
fb_percentiles_copy['likes_class'].value_counts()

low     169715
med     167276
high    165823
Name: likes_class, dtype: int64

In [13]:
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

/Users/harishprabhala/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [15]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import RegexpTokenizer
import re
import gensim.parsing.preprocessing as process

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/harishprabhala/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
lemmatizer = WordNetLemmatizer()

In [17]:
def preprocess (line):
    line = re.sub(r"http\S+", "", line)
    line = line.lower()
    line = process.remove_stopwords(line)
    line = process.strip_numeric(line)
    line = process.strip_punctuation(line)
    line = process.strip_short(line)
    line = process.strip_multiple_whitespaces(line)
    tokens = sent_tokenize(line)
    stemmed = [w for w in tokens if lemmatizer.lemmatize(w)] 
    return ' '.join(stemmed)

In [18]:
text = []
for w in fb_percentiles_copy['full_text']:
    text.append(preprocess(w))

In [19]:
X,y = text, fb_percentiles_copy['likes_class']

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
vectorizer = CountVectorizer()
X_1 = vectorizer.fit_transform(X)

In [83]:
X_new = SelectKBest(chi2, k=10000).fit_transform(X_1, y)

In [84]:
X_new.shape

(502814, 10000)

In [24]:
from sklearn.linear_model import LogisticRegression

In [85]:
X_train, X_test, Y_train, Y_test = train_test_split(X_new, y, test_size=0.33)

In [86]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((336885, 10000), (165929, 10000), (336885,), (165929,))

In [87]:
logit=LogisticRegression(multi_class='multinomial', solver='sag',C=0.1)
logit.fit(X_train,Y_train)
print ('Accuracy of test data: ',(accuracy_score(logit.predict(X_test), Y_test)))
print ('Accuracy of training data: ',(accuracy_score(logit.predict(X_train), Y_train)))
print("\n")
print ('Classification report summary of Multinomial Logistic Regression')
print(classification_report(logit.predict(X_test), Y_test))

Accuracy of test data:  0.489034466549
Accuracy of training data:  0.521097703964


Classification report summary of Multinomial Logistic Regression
             precision    recall  f1-score   support

       high       0.58      0.52      0.55     61437
        low       0.58      0.51      0.54     63578
        med       0.30      0.41      0.35     40914

avg / total       0.51      0.49      0.50    165929



In [88]:
clf2=MultinomialNB(alpha=0.01)
clf2.fit(X_train,Y_train)
print ('Accuracy of test data: ',(accuracy_score(clf2.predict(X_test), Y_test)))
print ('Accuracy of training data: ',(accuracy_score(clf2.predict(X_train), Y_train)))
print("\n")
print ('Classification report summary of MultinomialNB with alpha = 0.01:')
print(classification_report(clf2.predict(X_test), Y_test))

Accuracy of test data:  0.483670726636
Accuracy of training data:  0.497908781929


Classification report summary of MultinomialNB with alpha = 0.01:
             precision    recall  f1-score   support

       high       0.62      0.49      0.55     68662
        low       0.55      0.52      0.53     59675
        med       0.28      0.41      0.33     37592

avg / total       0.52      0.48      0.50    165929

